In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

import pandas as pd
import time



In [2]:
# 신한라이프 공시실 상품공시 자료중 판매중 상품 크롤링
def scrape_shinhan_life_products_on_sale(driver):
    data0 = []

    try:
        # 1. 페이지 접속
        driver.get("https://www.shinhanlife.co.kr/hp/cdhi0030.do")
        print("공시실 상품공시 판매중 웹사이트에 접속했습니다.")

        current_page = 1
        while True:
            print(f"\n현재 페이지: {current_page}")
            
            wait = WebDriverWait(driver, 30)
            
            # 2. 모든 '상세내용 보기' 버튼 클릭
            # print("모든 '상세내용 보기' 버튼을 찾아 클릭합니다...")
            detail_buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//button[contains(@class, "icoBtn_acco") and contains(@class, "accoBtn")]')))
            print(f"총 {len(detail_buttons)}개의 버튼을 찾았습니다.")
            
            for i, button in enumerate(detail_buttons, 1):
                try:
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(0.5)
                except Exception as e:
                    print(f"버튼 {i} 클릭 중 오류 발생: {e}")

            print(f"총 {len(detail_buttons)}개의 버튼 클릭을 완료했습니다.")

            time.sleep(5)  # 모든 상세 정보가 로드될 때까지 대기
            
            # 3. 테이블 찾기
            # print("테이블을 찾는 중...")
            table = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
            # print("테이블을 찾았습니다.")
            
            # 4. 테이블의 각 행을 읽어들이기
            rows = table.find_elements(By.TAG_NAME, 'tr')
            print(f"총 {len(rows)}개의 행을 찾았습니다.")

            last_category = ""  # 마지막으로 처리된 분류를 저장할 변수

            # 5. 각 행의 컬럼 내용을 분리하여 읽어들이기
            for index, row in enumerate(rows[1:], start=1):  # 헤더 행 제외
                # print(f"  행 {index} 처리 중...")
                cells = row.find_elements(By.TAG_NAME, 'td')
                if cells:
                    category = cells[0].text.strip()
                    if category:
                        last_category = category
                    else:
                        category = last_category
                    
                    product_info = cells[1].find_element(By.CLASS_NAME, 'accoHead')
                    
                    # 상품명에서 불필요한 텍스트 제거
                    product_name = product_info.find_element(By.CLASS_NAME, 'cell_1').text.strip()
                    product_name = product_name.split('\n')[0]  # 첫 번째 줄만 사용
                    
                    # 판매기간 추출 및 정제
                    sale_periods = []
                    for period in product_info.find_elements(By.CLASS_NAME, 'cell_2'):
                        period_text = period.text.strip()
                        if '~' in period_text:
                            sale_periods.append(period_text)
                    
                    # URL 추출 및 변환 함수
                    def get_url(xpath):
                        elements = product_info.find_elements(By.XPATH, xpath)
                        if elements:
                            url = elements[0].get_attribute('data-url')
                            if url:
                                if url.startswith('/repo/DigitalPlattform/'):
                                    url = '/bizxpress/' + url[23:]
                                full_url = f"https://www.shinhanlife.co.kr{url}"
                                return full_url
                        return 'X'
                    
                    summary = get_url(".//button[contains(@data-title, '요약서')]")
                    method = get_url(".//button[contains(@data-title, '방법서')]")
                    terms = get_url(".//button[contains(@data-title, '약관')]")
                    
                    # 각 판매기간마다 데이터 추가
                    for sale_period in sale_periods:
                        row_data = [
                            "판매중",  # 판매구분
                            "신한라이프",  # 판매사
                            category,
                            product_name,
                            sale_period,
                            summary,
                            method,
                            terms
                        ]
                        data0.append(row_data)
                
                # 분류가 비어있는 경우 이전 행의 분류로 채우기
                if not category and data0:
                    data0[-1][2] = data0[-2][2]  # 인덱스 2는 '분류' 컬럼
            
            print(f"현재까지 총 {len(data0)}개의 행을 수집했습니다.")
            
            # 6. 다음 페이지로 이동
            try:
                wait = WebDriverWait(driver, 10)
                pagination = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'paging')))
                page_links = pagination.find_elements(By.XPATH, './/ul/li[@class=""]/a')
                max_pages = len(page_links) + 1  # 현재 페이지도 포함하므로 +1
                
                print(f"총 {max_pages}개의 페이지가 있습니다.")
                
                if current_page < max_pages:
                    next_page_link = page_links[current_page - 1]  # current_page는 1부터 시작하므로 인덱스 조정
                    print(f"{current_page + 1}페이지로 이동합니다.")
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_page_link)
                    time.sleep(1)  # 스크롤 후 잠시 대기
                    driver.execute_script("arguments[0].click();", next_page_link)
                    current_page += 1
                    time.sleep(5)  # 페이지 로딩 대기
                else:
                    # 현재 페이지 세트의 마지막 페이지에 도달한 경우
                    next_button = pagination.find_element(By.CLASS_NAME, 'icoBtn_next')
                    disabled_attr = next_button.get_attribute('disabled')

                    if disabled_attr is None:
                        print("다음 페이지 세트로 이동합니다.")
                        driver.execute_script("arguments[0].click();", next_button)
                        current_page = 1  # 페이지 카운터 리셋 (새 페이지 세트의 첫 페이지로)
                        time.sleep(5)  # 페이지 로딩 대기
                    else:
                        print("모든 페이지의 크롤링을 완료했습니다.")
                        break  # 크롤링 종료
            except (NoSuchElementException, TimeoutException) as e:
                print(f"페이지 링크를 찾을 수 없습니다: {e}")
                print("크롤링을 종료합니다.")
                break
            except Exception as e:
                print(f"페이지 이동 중 예상치 못한 오류 발생: {e}")
                print("크롤링을 종료합니다.")
                break
                    
    except Exception as e:
                print(f"오류가 발생했습니다: {e}")

    # 7. 판다스 데이터프레임으로 변환
    df1 = pd.DataFrame(data0, columns=["판매구분", "판매사", "분류", "상품명", "판매기간", "요약서", "방법서", "약관"])
    # print(df1.head())  # 처음 5개 행 출력
    
    return df1, driver

In [4]:
# 신한라이프 공시실 상품공시 자료중 판매중지 상품 크롤링
def scrape_shinhan_life_products_discontinued(driver):
    data1 = []

    try:
        # 1. 페이지 접속
        driver.get("https://www.shinhanlife.co.kr/hp/cdhi0040t01.do")
        print("\n\n\n공시실 상품공시 판매중지 웹사이트에 접속했습니다.\n\n\n")

        # 탭 목록 가져오기
        tabs = driver.find_elements(By.CSS_SELECTOR, "#tabList li")
        print("tabs:", tabs)
        for tab_index, tab in enumerate(tabs):
            print(f"\n{tab.text} 탭 처리 중...")
            
            # 탭 클릭
            driver.execute_script("arguments[0].click();", tab)
            time.sleep(5)  # 탭 전환 대기

            current_page = 1
            while True:
                print(f"\n현재 페이지: {current_page}")
                
                wait = WebDriverWait(driver, 30)
                
                # 2. 모든 '상세내용 보기' 버튼 클릭
                # print("모든 '상세내용 보기' 버튼을 찾아 클릭합니다...")
                detail_buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//button[contains(@class, "icoBtn_acco") and contains(@class, "accoBtn")]')))
                print(f"총 {len(detail_buttons)}개의 버튼을 찾았습니다.")
                
                for button in detail_buttons:
                    try:
                        driver.execute_script("arguments[0].click();", button)
                        time.sleep(0.5)
                    except Exception as e:
                        print(f"버튼 클릭 중 오류 발생: {e}")

                print(f"총 {len(detail_buttons)}개의 버튼 클릭을 완료했습니다.")
                
                time.sleep(7)  # 모든 상세 정보가 로드될 때까지 대기
                
                # 3. 테이블 찾기
                # print("테이블을 찾는 중...")
                table = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
                # print("테이블을 찾았습니다.")
                
                # 4. 테이블의 각 행을 읽어들이기
                rows = table.find_elements(By.TAG_NAME, 'tr')
                print(f"총 {len(rows)}개의 행을 찾았습니다.")

                last_category = ""  # 마지막으로 처리된 분류를 저장할 변수

                # 5. 각 행의 컬럼 내용을 분리하여 읽어들이기
                for index, row in enumerate(rows[1:], start=1):  # 헤더 행 제외
                    # print(f"  행 {index} 처리 중...")
                    cells = row.find_elements(By.TAG_NAME, 'td')
                    if cells:
                        category = cells[0].text.strip()
                        if category:
                            last_category = category
                        else:
                            category = last_category
                        
                        product_info = cells[1].find_element(By.CLASS_NAME, 'accoHead')
                        
                        # 상품명에서 불필요한 텍스트 제거
                        product_name = product_info.find_element(By.CLASS_NAME, 'cell_1').text.strip()
                        product_name = product_name.split('\n')[0]  # 첫 번째 줄만 사용
                        
                        # 판매기간 추출 및 정제
                        sale_periods = []
                        for period in product_info.find_elements(By.CLASS_NAME, 'cell_2'):
                            period_text = period.text.strip()
                            if '~' in period_text:
                                sale_periods.append(period_text)
                        
                        # URL 추출 및 변환 함수
                        def get_url(xpath):
                            elements = product_info.find_elements(By.XPATH, xpath)
                            if elements:
                                url = elements[0].get_attribute('data-url')
                                if url:
                                    if url.startswith('/repo/DigitalPlattform/'):
                                        url = '/bizxpress/' + url[23:]
                                    full_url = f"https://www.shinhanlife.co.kr{url}"
                                    return full_url
                            return 'X'
                        
                        summary = get_url(".//button[contains(@data-title, '요약서')]")
                        method = get_url(".//button[contains(@data-title, '방법서')]")
                        terms = get_url(".//button[contains(@data-title, '약관')]")
                        
                        # 각 판매기간마다 데이터 추가
                        for sale_period in sale_periods:
                            row_data = [
                                "판매중지",  # 판매구분
                                tab.text,  # 판매사 (탭 이름으로 설정)
                                category,
                                product_name,
                                sale_period,
                                summary,
                                method,
                                terms
                            ]
                            data1.append(row_data)
                    
                    # 분류가 비어있는 경우 이전 행의 분류로 채우기
                    if not category and data1:
                        data1[-1][2] = data1[-2][2]  # 인덱스 2는 '분류' 컬럼
                
                print(f"현재까지 총 {len(data1)}개의 행을 수집했습니다.")
                
                # 6. 다음 페이지로 이동
                try:
                    wait = WebDriverWait(driver, 10)
                    pagination = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'paging')))
                    page_links = pagination.find_elements(By.XPATH, './/ul/li[@class=""]/a')
                    max_pages = len(page_links) + 1  # 현재 페이지도 포함하므로 +1
                    
                    print(f"총 {max_pages}개의 페이지가 있습니다.")
                    
                    if current_page < max_pages:
                        next_page_link = page_links[current_page - 1]  # current_page는 1부터 시작하므로 인덱스 조정
                        print(f"{current_page + 1}페이지로 이동합니다.")
                        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_link)
                        time.sleep(1)  # 스크롤 후 잠시 대기
                        driver.execute_script("arguments[0].click();", next_page_link)
                        current_page += 1
                        time.sleep(5)  # 페이지 로딩 대기
                    else:
                        # 현재 페이지 세트의 마지막 페이지에 도달한 경우
                        next_button = pagination.find_element(By.CLASS_NAME, 'icoBtn_next')
                        disabled_attr = next_button.get_attribute('disabled')

                        if disabled_attr is None:
                            print("다음 페이지 세트로 이동합니다.")
                            driver.execute_script("arguments[0].click();", next_button)
                            current_page = 1  # 페이지 카운터 리셋 (새 페이지 세트의 첫 페이지로)
                            time.sleep(5)  # 페이지 로딩 대기
                        else:
                            print("모든 페이지의 크롤링을 완료했습니다.")
                            break  # 크롤링 종료
                except (NoSuchElementException, TimeoutException) as e:
                    print(f"페이지 링크를 찾을 수 없습니다: {e}")
                    print("크롤링을 종료합니다.")
                    break
                except Exception as e:
                    print(f"페이지 이동 중 예상치 못한 오류 발생: {e}")
                    print("크롤링을 종료합니다.")
                    break

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

    # 7. 판다스 데이터프레임으로 변환
    df2 = pd.DataFrame(data1, columns=["판매구분", "판매사", "분류", "상품명", "판매기간", "요약서", "방법서", "약관"])
    # print(df2.head())  # 처음 5개 행 출력
    
    return df2, driver

In [5]:
from webdriver_manager.chrome import ChromeDriverManager
# 메인 실행 부분
chrome_options = Options()
chrome_options.add_argument("--headless")  # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# service = Service('/usr/local/bin/chromedriver')
# driver = webdriver.Chrome(service=service, options=chrome_options)

# ChromeDriverManager가 자동으로 알맞은 버전의 드라이버를 설치합니다
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    df1, driver = scrape_shinhan_life_products_on_sale(driver)
    df2, driver = scrape_shinhan_life_products_discontinued(driver)
    
    # 두 데이터프레임 합치기
    combined_df = pd.concat([df1, df2], ignore_index=True)
    # combined_df = df1
    
    # 합친 데이터프레임을 엑셀로 저장
    output_file = "shinhan_life_products_combined.xlsx"
    combined_df.to_excel(output_file, index=False)
    
    print(f"데이터를 {output_file} 파일로 저장했습니다.")
    print(f"총 {len(combined_df)} 개의 행이 저장되었습니다.")
    print(f"판매 중 상품: 총 {len(df1)} 개의 행")
    print(f"판매 중지 상품: 총 {len(df2)} 개의 행")
    
    # 판매구분별 행 수 출력
    print("\n판매구분별 행 수:")
    print(combined_df['판매구분'].value_counts())
    
    # 판매사별 행 수 출력
    print("\n판매사별 행 수:")
    print(combined_df['판매사'].value_counts())

finally:
    driver.quit()
    print("브라우저를 종료했습니다.")

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 128
Current browser version is 124.0.6367.118 with binary path /opt/google/chrome/chrome
Stacktrace:
#0 0x563da7e8e2da <unknown>
#1 0x563da7b5c200 <unknown>
#2 0x563da7b9ae74 <unknown>
#3 0x563da7b99de7 <unknown>
#4 0x563da7b94c8f <unknown>
#5 0x563da7b90058 <unknown>
#6 0x563da7bdca7e <unknown>
#7 0x563da7bdc296 <unknown>
#8 0x563da7bd0673 <unknown>
#9 0x563da7b9e473 <unknown>
#10 0x563da7b9f47e <unknown>
#11 0x563da7e550db <unknown>
#12 0x563da7e59071 <unknown>
#13 0x563da7e419d5 <unknown>
#14 0x563da7e59bf2 <unknown>
#15 0x563da7e26b6f <unknown>
#16 0x563da7e7d248 <unknown>
#17 0x563da7e7d417 <unknown>
#18 0x563da7e8d0cc <unknown>
#19 0x7fc8438a1ac3 <unknown>
